# Python Technologist Application Test

## Problem 1


[This is](https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D9%85%D8%A6%D8%A9_%D8%B1%D9%88%D8%A7%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9) a Wikipedia webpage listing the 100 best Arabic novels according to the Arab Writers Union. In case it did not open for any reason try [this](https://www.marefa.org/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D8%A7%D9%84%D9%83%D8%AA%D8%A8_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9) alternative link for the same info 

Using Python, do the following:

1. Scrap the webapge to get the books table and write it to excel file, Keeping all the content from the HTML table including Hyper-links if any.
2. Write the content to a Google sheet 


Write your code in the following cell. You are free to add cells as much as you need.

### Problem 1 Answer
---

In [1]:
import requests
from bs4 import BeautifulSoup

# Declaring lists used for storing output data from web scraping
novel_names_list = []
novel_authors_list = []
novel_country_list = []
novel_names_urls = []
novel_authors_urls = []
novel_country_urls = []


In [2]:
url_id = '%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D9%85%D8%A6%D8%A9_%D8%B1%D9%88%D8%A7%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9'
url = f'https://ar.wikipedia.org/wiki/{url_id}'


In [3]:
def web_scraper(web_url):
    requested_page = requests.get(web_url) # Get Data From WebPage

    html_content = BeautifulSoup(requested_page.content, 'html.parser') # Get Html Content from webpage data

    table_rows = html_content.find_all('tr') # Get all table rows from Html Content

    """
        Loops on every row to get each cell data

        First cell -> Novel Name,
        Second cell -> Author Name,
        Third cell -> Author's Country

        Loops on every cell to get hyperlink of each one

        Appends each data to its relevant declared list
    """ 
    for row in table_rows[1:-1]:
        
        row_cell = row.find_all('a') # Html of wikipidia page includes each table cell in an anchor tag 

        #output is a list that contains three elements one for each cell in every row 
        novel_names_list.append(row_cell[0].text)
        novel_authors_list.append(row_cell[1].text)
        novel_country_list.append(row_cell[2].text)
        
        novel_names_urls.append(
            f"https://ar.wikipedia.org{row_cell[0]['href']}")
        novel_authors_urls.append(
            f"https://ar.wikipedia.org{row_cell[1]['href']}")
        
        novel_country_urls.append(
            f"https://ar.wikipedia.org{row_cell[2]['href']}")


web_scraper(url)


In [4]:
import pandas as pd

In [5]:
"""
    Creates a pandas dataframe using the output data from webscraping
    Headers of dataframe are -> الروايه,المؤلف,البلد like the headers in the wikipidia table 
    each row in data frame contain -> novel name, novel author, novel country
    head of dataframe : 
               الروايه                 المؤلف          البلد                 
1               الثلاثية              نجيب محفوظ          مصر
2            البحث عن وليد مسعود    جبرا إبراهيم جبرا      فلسطين
3               شرف            صنع الله إبراهيم        مصر
4               الحرب في بر مصر          يوسف القعيد         مصر
5             رجال في الشمس            غسان كنفاني        فلسطين
...

This function will be used while making excel sheet and google sheet
""" 
def create_dataframe():
    df = pd.DataFrame({'الروايه': novel_names_list,
                       'المؤلف': novel_authors_list,
                       'البلد': novel_country_list})
    
    df.index += 1 # shifts dataframe so that it starts from 1 not 0 such as the wikipidia table
    
    return df

#Run these only if you need to print the dataframe 
# data=create_dataframe()
# print (data.head) 

In [6]:
# Used to format excel sheet to fit arabic content 
def format_excel_sheet(workbook, worksheet):
    format_right_to_left = workbook.add_format({'reading_order': 2}) # Add the cell formats.

    worksheet.right_to_left() # Change the direction for the worksheet.

    worksheet.set_column('B:D', 20, format_right_to_left)# Make the column wider for visibility and add the reading order format.


The following function creates an excel sheet without the hyperlinks in the wikipidia page, however in order to create an excel sheet with all the hyperlinks in the table I needed to add a hyperlink function in each dataframe element as the following: 

'=HYPERLINK("https://www.google.com", "google")'

Which I did and the output was the excel sheet __Final_with_links.xlsx__ which includes clickable cells to the refrenced url

However, I was unable to read this excel sheet (__Final_with_links.xlsx__) in question 2 although I searched alot and even asked on stack over flow why im unable to read it after creating it -> [here](https://stackoverflow.com/questions/73360749/reading-an-excel-sheet-containing-hyperlinks-using-pythons-pandas-read-excel/73362816#73362816)
But I was unable to solve the problem, I was also unable to create hyperlinks in the excel sheet with any other way although I tried many ways.


In [7]:
'''
Main function that creates excel sheet
After calling this function, Final_without_links.xlsx file will be created in the same directory
Excel sheet is created from dataframe containing all data in wikipidia table
'''
def create_excel_from_dataframe():
    df = create_dataframe()

    writer = pd.ExcelWriter('Final_without_links.xlsx', engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine.

    df.to_excel(writer, sheet_name='Sheet1') # Convert the dataframe to an XlsxWriter Excel object.

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    format_excel_sheet(workbook, worksheet)

    writer.save()


create_excel_from_dataframe()


Source code in file googleFile is from [this link](https://learndataanalysis.org/creating-google-sheets-file-google-sheets-api-in-python-part-2/)

Client secret file can be downloaded from [google sheet api](https://developers.google.com/sheets/api) after being authenticated on google cloud 

[This](https://docs.google.com/spreadsheets/d/1295hSdHmQwGPOMSG4YKh-6PzIvTqYXOg2b4MCpzvxC0/edit#gid=982029778) is the __link to the google sheet__ i created containing all the data from the wikipidia table

In [8]:
from googleFile import Create_Service

In [9]:
# Function creates google sheet service with the sheet properties
def create_google_sheet():
    # Parameters needed to create google sheet service
    CLIENT_SECRET_FILE = 'client_secret_json.json'
    API_NAME = 'sheets'
    API_VERSION = 'v4'
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    google_sheet_service = Create_Service(
        CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
    
    # Specifying sheet properties (title, direction)
    sheet_body = {
        'properties': {
            'title': 'Best Arabic Novels',
        },
        'sheets': [
            {
                'properties': {
                    'title': 'Sheet 1',
                    'rightToLeft': True
                }
            }
        ]
    }
    sheet_file = google_sheet_service.spreadsheets().create(body=sheet_body).execute()
    return google_sheet_service, sheet_file

In [10]:
'''
Function appends dataframe content to google sheet created
Each run will create a different google sheet and print the url of it
'''
def add_df_to_google_sheet():
    df = create_dataframe()
    google_sheet_service,sheet_file=create_google_sheet()
    
    # Adding data to google sheet
    google_sheet_service.spreadsheets().values().append(
        spreadsheetId=sheet_file['spreadsheetId'],
        valueInputOption='RAW',
        range='DF1:A1',
        body=dict(
            majorDimension='ROWS',
            values=df.T.reset_index().T.values.tolist()),
    ).execute() 
    print(sheet_file['spreadsheetUrl'])
    
add_df_to_google_sheet()

client_secret_json.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
sheets service created successfully
https://docs.google.com/spreadsheets/d/1y4FNnVMK4Htu_hFGHEcI8-4j1BSO-5OCe2qoWMnJTUo/edit


## Problem 2 

Create REST APIs in Python using Flask to read (Get) and write (Post, Delete, Put) the local excel file from the previous problem. Please make sure to bundle all the API dependencies to be uasble. Deploying the API to Heroku would be a big plus.

Write your code in the following cell. You are free to add cells as much as you need.

### Problem 2 Answer
------
Please run file [this file](novel_api.py) in order to run the flask application/server, the following cells contain the code of the flask application but __will not run the server__

In [11]:
from flask_restful import Api, reqparse, abort
from flask import Flask


In [12]:
#Reads excel sheet and converts it to a Dataframe
def convert_excel_to_df():
    df = pd.read_excel('Final_without_links.xlsx')
    del df['Unnamed: 0'] #Remove unneeded column "unnamed:0" from dataframe
    return df

In [13]:
def format_excel(writer, df):
    df.to_excel(writer, sheet_name='Sheet1')

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    format_excel_sheet(workbook,worksheet)
    writer.save()

In [14]:
# Create flask application and API
app = Flask(__name__)
api = Api(app)


In [15]:
novel_args = reqparse.RequestParser() #Define novel arguments

#Add three novel arguments (novel name, author name, author country)
novel_args.add_argument(
    "الروايه", type=str, help="Name of the novel", required=True)
novel_args.add_argument(
    "المؤلف", type=str, help="Name of the author", required=True)
novel_args.add_argument(
    "البلد", type=str, help="Country of author", required=True)


In [16]:
# If index sent in request is larger than dataframe length -> abort without crashing api
def abort_if_index_out_of_range(index, df):
    if index > len(df):
        abort(404, message="Novel Index out of range")


In [17]:
# Home page of flask app
@app.route("/")
def index():
    return ("Best-Arabic-Novels Api")

In [18]:
'''
Rest Api to read/get data from local excel sheet
takes novel id as a path variable
returns json response with data requested
sends status code 200 at successful get requests
'''

@app.route('/Novel/<int:novel_id>', methods=['GET'])
def get_novel(novel_id):
    df = convert_excel_to_df()
    abort_if_index_out_of_range(novel_id, df)

    # minus 1 from id in dataframe to shift index from zero to match id in excel sheet
    requested_novel_name = f"{ (df.get('الروايه')[novel_id-1])}"
    requested_novel_author = f"{ (df.get('المؤلف')[novel_id-1])}"
    requested_novel_country = f"{ (df.get('البلد')[novel_id-1])}"
    return {("الروايه"): requested_novel_name,
            ("المؤلف"): requested_novel_author,
            ("البلد"): requested_novel_country}


In [19]:
'''
Rest Api to post data to local excel sheet
takes three arguments -> novel name, author name, author country
returns json response with posted data and status code
sends status code 201 at successful post requests
'''

@app.route('/Novel', methods=['POST'])
def post_novel():
    args = novel_args.parse_args()

    df = convert_excel_to_df()
    df.loc[len(df), df.columns] = args['الروايه'], args['المؤلف'], args['البلد']
    df.index = df.index + 1  # shifting index
    writer = pd.ExcelWriter(
        'Final_without_links.xlsx', engine='xlsxwriter')
    format_excel(writer, df)

    return {("الروايه"): (args['الروايه']),
            ("المؤلف"):  (args['المؤلف']),
            ("البلد"):  (args['البلد'])}, 201


In [20]:
'''
Rest Api to put data to local excel sheet at refrenced novel id
takes refrenced novel id as path variable
takes three arguments -> novel name, author name, author country
returns json response with data put and status code
sends status code 201 at successful post requests
'''

@app.route('/Novel/<int:novel_id>', methods=['PUT'])
def put_novel(novel_id):
    args = novel_args.parse_args()
    df = convert_excel_to_df()
    abort_if_index_out_of_range(novel_id, df)

    df.loc[novel_id-1] = args['الروايه'], args['المؤلف'], args['البلد']
    df.index = df.index + 1  # shifting index
    writer = pd.ExcelWriter(
        'Final_without_links.xlsx', engine='xlsxwriter')
    format_excel(writer, df)

    return {("الروايه"):  (args['الروايه']),
            ("المؤلف"):  (args['المؤلف']),
            ("البلد"):  (args['البلد'])}, 201


In [21]:
'''
Rest Api to delete data from local excel sheet at refrenced novel id
takes refrenced novel id as path variable
returns status code
sends status code 204 at successful delete requests
'''

@app.route('/Novel/<int:novel_id>', methods=['DELETE'])
def delete_novel(novel_id):
    df = convert_excel_to_df()
    abort_if_index_out_of_range(novel_id, df)

    df = df.drop(df.index[novel_id-1])  # drops refrenced id row from dataframe
    df = df.reset_index(drop=True)  # resets indices of dataframe
    df.index = df.index + 1
    writer = pd.ExcelWriter(
        'Final_without_links.xlsx', engine='xlsxwriter')
    format_excel(writer, df)
    return '', 204


### Test cases for local API 
After running [novel api file](novel_api.py) the local server will be http://127.0.0.1:5000/

This Api can __get__, __post__ ,__put__ and __delete__ from the local excel sheet.

The following test cases applied on the local api do the following:

1. Get first element
2. Delete first element
3. Get first element again after index 1 has been deleted
4. Put request to edit index 50
5. Gets index 50
6. Posts to the end of the local excel sheet

Cases tested on __postman__ or __jupyter notebook__ dont need arabic words to be reshaped

Cases tested in __terminal__ need arabic words to be reshaped by passing the arabic word to the function Arabic_reshape()

In [22]:
def test_cases_for_api(base_url):
    novel_id=1
    response_get=requests.get(base_url+f"Novel/{novel_id}") #Gets first novel
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")

    response_del=requests.delete(base_url+f"Novel/{novel_id}") #Deletes first novel
    print(f"Status code for deleting novel {novel_id} is {response_del.status_code}")

    response_get=requests.get(base_url+f"Novel/{novel_id}") #Gets first novel after we deleted novel at index 1
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")


    novel_id=50
    response_put=requests.put(base_url+f"Novel/{novel_id}",{'الروايه':'الحرافيش','المؤلف':'نجيب محفوظ','البلد':'مصر'}) #edits novel at index 50
    print(f"Put response for novel {novel_id} = {response_put.json()} with status code {response_put.status_code}")

    response_get=requests.get(base_url+f"Novel/{novel_id}") #Gets novel at index 50
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")


    response_post=requests.post(base_url+"Novel",{'الروايه':'ثرثرة فوق النيل','المؤلف':'نجيب محفوظ','البلد':'مصر'}) #posts new novel at the end of excel file
    print(f"Put response for novel {novel_id} = {response_post.json()} with status code {response_post.status_code}")
    

base_url='http://127.0.0.1:5000/'
test_cases_for_api(base_url)

Get response for novel 1 = {'البلد': 'مصر', 'الروايه': 'الثلاثية', 'المؤلف': 'نجيب محفوظ'} and status code is 200
Status code for deleting novel 1 is 204
Get response for novel 1 = {'البلد': 'فلسطين', 'الروايه': 'البحث عن وليد مسعود', 'المؤلف': 'جبرا إبراهيم جبرا'} and status code is 200
Put response for novel 50 = {'البلد': 'مصر', 'الروايه': 'الحرافيش', 'المؤلف': 'نجيب محفوظ'} with status code 201
Get response for novel 50 = {'البلد': 'مصر', 'الروايه': 'الحرافيش', 'المؤلف': 'نجيب محفوظ'} and status code is 200
Put response for novel 50 = {'البلد': 'مصر', 'الروايه': 'ثرثرة فوق النيل', 'المؤلف': 'نجيب محفوظ'} with status code 201


### Test cases for Heroku API 
API Deployed to heroku and application link is: https://heroku-novels-demo.herokuapp.com/

This Api can __get__ and __delete__ from excel sheet uploaded in [github repository](https://github.com/Meirna-kamal/heroku-app).

__post__ and __put__ functions have been deployed but they return a bad request (status code 400), I tried searching for the reason but unfortuntly i was unable to solve the issue

The following test cases applied on the deployed heroku api do the following:

1. Get first element
2. Delete first element
3. Get first element again after index 1 has been deleted
4. Put request to edit index 50 -> status code 400 (bad request)
5. Gets index 50
6. Posts to the end of the local excel sheet -> status code 400 bad request

In [23]:
import requests

def test_cases_for_api(base_url):
    novel_id=1
    response_get=requests.get(base_url+f"Novel/{novel_id}")
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")

    response_del=requests.delete(base_url+f"Novel/{novel_id}")
    print(f"Status code for deleting novel {novel_id} is {response_del.status_code}")

    response_get=requests.get(base_url+f"Novel/{novel_id}")
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")

    novel_id=50
    response_put=requests.put(base_url+f"Novel/{novel_id}",{'الروايه':'الحرافيش','المؤلف':'نجيب محفوظ','البلد':'مصر'})
    print(f"status code {response_put.status_code}")

    response_get=requests.get(base_url+f"Novel/{novel_id}")
    print (f"Get response for novel {novel_id} = {response_get.json()} and status code is {response_get.status_code}")

    response_post=requests.post(base_url+"Novel",{'الروايه':'ثرثرة فوق النيل','المؤلف':'نجيب محفوظ','البلد':'مصر'})
    # print(f"Put response for novel {novel_id} = {response_post.json()} with status code {response_post.status_code}")
    print(f"status code {response_post.status_code}")
    

base_url='https://heroku-novels-demo.herokuapp.com/'
test_cases_for_api(base_url)

Get response for novel 1 = {'البلد': 'مصر', 'الروايه': 'الثلاثية', 'المؤلف': 'نجيب محفوظ'} and status code is 200
Status code for deleting novel 1 is 204
Get response for novel 1 = {'البلد': 'فلسطين', 'الروايه': 'البحث عن وليد مسعود', 'المؤلف': 'جبرا إبراهيم جبرا'} and status code is 200
status code 400
Get response for novel 50 = {'البلد': 'مصر', 'الروايه': 'أيام الإنسان السبعة', 'المؤلف': 'عبد الحكيم قاسم'} and status code is 200
status code 400


## Problem 3

Write a tool to create a PDF cover for the books, following the attached example "book-cover-sample.pdf". Considering the following:

    1. The QR code should embed the book hyperlink from Wikipedia.
    2. The QR code should be clickable to let the users acess it by clicking on it.
    3. Include all the covers in one directory and compress it in ZIP format.
    

Write your code in the following cell. You are free to add cells as much as you need.

### Problem 3 Answer
---

In [24]:
import qrcode

In [25]:
# Function that creates QR code for each novel
def create_qr_code(novel_index):
    qr = qrcode.QRCode()
    novel_url = novel_names_urls[novel_index]
    qr.add_data(novel_url) # Adds link of novel to qr code to route directly to link when qr code is scanned
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')
    img.save('qr.png')


In [26]:
import os

In [27]:
#Deletes qr image after each iteration to create each pdf as img is no longer needed
def delete_qr_image_from_directory():
    os.remove("qr.png")

In [28]:
import shutil

In [29]:
#Create Novel pdfs folder to save pdfs
def create_pdfs_folder():
    os.makedirs("novel_pdfs")

In [30]:
# Compresses Novel pdfs folder
def compress_novel_folder(zip_name,folder_name):
    shutil.make_archive(zip_name, 'zip', folder_name)

In [31]:
import arabic_reshaper
from bidi.algorithm import get_display

In [32]:
'''
Corrects arabic letters direction and character shape
Used for printing in terminal and in creating pdfs

example ->
When printing ذهب الطالب الى المدرسة
ط°ظ‡ط¨ ط§ظ„ط·ط§ظ„ط¨ ط§ظ„ظ‰ ط§ظ„ظ…ط¯ط±ط³ط© -> ذهب الطالب الى المدرسة
'''
def arabic_reshape(arabic_word):
    shape_corrected_word = arabic_reshaper.reshape(arabic_word)
    direction_corrected_word = get_display(shape_corrected_word)
    return direction_corrected_word

In [33]:
from fpdf import FPDF
from warnings import filterwarnings

In [35]:

'''
Main Function that creates pdfs
Creates zip folder(Compressed_novels.zip) containing a pdf file for each novel
Each pdf contains a clickable qr code that upon clicking or scanning redirects to the novels webpage on wikipidia 
''' 
def create_pdfs():
    create_pdfs_folder()
    for i in range(len(novel_names_list)): #loops on each novel to create its pdf
        create_qr_code(i)
        pdf = FPDF() 
        pdf.add_page() #Adding blank page to pdf

        pdf.add_font("NotoSansArabic", '', 'NotoSansArabic-Regular.ttf', uni=True) #Adding font suitable for arabic characters
        pdf.set_font('NotoSansArabic', size=40) #Applying font and font size to pdf

        pdf.image('qr.png', 50, 60, 110, 110) #Adding Qr image to pdf
        pdf.link(58, 68, 95, 93, novel_names_urls[i]) #Adding a space that contains the novel link over the qr img in order to be clickable 

        # moving 20*10 cell 170 downwards, 85 right, and centering its content in its box 
        pdf.ln(170)
        pdf.cell(85)
        pdf.cell(20, 10, arabic_reshape(novel_names_list[i]), 0, 0, 'C') #Adding novel name in cell

        # moving 20*10 cell 20 downwards, 85 right, and centering its content in its box 
        pdf.ln(20)
        pdf.cell(85)
        pdf.cell(20, 10, arabic_reshape(novel_authors_list[i]), 0, 0, 'C') #Adding author name in cell

        pdf.output(f'novel_pdfs/{novel_names_list[i]}.pdf') #Saves pdf to novel_pdfs folder, with the name of the novel
        
        delete_qr_image_from_directory()
    compress_novel_folder('compressed_novels','novel_pdfs') #compressing novel_pdfs to compressed_novels
filterwarnings('ignore')
create_pdfs()